In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as s
from netCDF4 import Dataset
import os
from vis_io import get_vtu, write_vtu, get_vts, write_vts, get_mesh
import vtk
from data.data import SphericalDataset
from vtkmodules.util import numpy_support
import scipy.optimize as op

In [15]:
ds = SphericalDataset('../data/earth_nopole.npy', [0,1,2], [3,4,5])

In [5]:
a = np.arange(1,10).reshape(3,3)

In [19]:
np.linalg.det(np.identity(3))

1.0

In [19]:
next(iter(ds))

(tensor([  581.6089,     0.0000, -3436.1252]),
 tensor([3485.0000,    0.0000,  170.3930]))

In [26]:
# //
  # //		    6________7
  # //		   /|       /|
  # //		  / |      / |
  # //		4/_______5/  |
  # //		|  2|___ |___|3
  # //		|  /     |  /
  # //		| /      | /
  # //		|/_______|/
  # //		0        1
def getDiagCell(i, r, theta, phi):
  return np.array([
    [r[i], theta[i], phi[i]],
    [r[i+1], theta[i], phi[i]],
    [r[i], theta[i+1], phi[i]],
    [r[i+1], theta[i+1], phi[i]],
    [r[i], theta[i], phi[i+1]],
    [r[i+1], theta[i], phi[i+1]],
    [r[i], theta[i+1], phi[i+1]],
    [r[i+1], theta[i+1], phi[i+1]],
  ])

def isSorted(arr, verbose=False):
  '''
  check if sorted either ascend or decend
  '''
  isascend = True
  isdecend = True
  for i in range(len(arr)-1):
    if isascend and arr[i] >= arr[i+1]:
      if verbose: print(f'Not ascend: arr[{i}] >= arr[{i+1}] - {arr[i]} >= {arr[i+1]}')
      isascend = False
    
    if isdecend and arr[i] <= arr[i+1]:
      if verbose: print(f'Not decend: arr[{i}] <= arr[{i+1}] - {arr[i]} <= {arr[i+1]}')
      isdecend = False
    
    if not isascend and not isdecend:
      return False
  if isascend: print('ascending sorted')
  else: print('decending sorted')
  
  return True

def deg2rad(deg):
  return deg * np.pi / 180

# for numerical isse
def mysin(a):
  val = np.sin(a)
  val = close_round(val, 0)
  val = close_round(val, 1)
  return val

def mycos(a):
  val = np.cos(a)
  val = close_round(val, 0)
  val = close_round(val, 1)
  return val

def close_round(val, test_val, abs_bounds=1e-12):
  isclose = np.abs(test_val - val) < abs_bounds
  # print(isclose)
  if isinstance(val, float) or isinstance(val, int):
    val_cp = test_val if isclose else val
  else:
    val_cp = val.copy()
    val_cp[isclose] = test_val
  return val_cp

def sph2car(r, theta, phi):
  # x_coef = np.sin(phi)*np.cos(theta)
  # y_coef = np.around(np.sin(phi)*np.sin(theta), decimals=10)
  # z_coef = np.around(np.cos(phi), decimals=10)
  x = r*mysin(phi)*mycos(theta)
  y = r*mysin(phi)*mysin(theta)
  z = r*mycos(phi)
  
  return [x,y,z]

def car2sph(x, y, z):
  # assert (x or y) != 0

  r = np.sqrt(x*x + y*y + z*z)
  theta = np.arctan(y/x)
  phi = np.arctan(np.sqrt(x*x + y*y) / z)

  # if x > 0:
  #   phi = np.arctan(y/x)
  # elif x < 0 and y >= 0:
  #   phi = np.arctan(y/x)+np.pi
  # elif x < 0 and y < 0:
  #   phi == np.arctan(y/x)-np.pi
  # elif x == 0 and y > 0:
  #   phi = np.pi/2
  # elif x == 0 and y < 0:
  #   phi = -np.pi/2

  return [r,theta,phi]

In [3]:
mt = Dataset("/fs/project/PAS0027/mantle/data/spherical010.nc", format="netcdf4")
# r:
# lon: [0, 360] theta (math)
# lat: [-90, 90] phi (math)
r = mt.variables['r'][:]
lon = mt.variables['lon'][:]
lat = mt.variables['lat'][:] + 90
temp = mt.variables['temperature'][:]

print('lon check sorted:')
isSorted(lon)
print('lat check sorted:')
isSorted(lat)
print('r check sorted:')
isSorted(r)

mt.close()

lon check sorted:
ascending sorted
lat check sorted:
decending sorted
r check sorted:
ascending sorted


In [4]:
# downsample a bit
r = r[:]
lon = lon[:]
lat = lat[range(10, len(lat)-10)]

In [193]:
lon = deg2rad(lon)
lat = deg2rad(lat)

In [67]:
lat = lat[None]

In [5]:
# output comp space
dims = [len(r), len(lon), len(lat)]
coords = get_mesh(r, lon, lat).reshape(-1, 3)
temp_val = temp.flatten()[:len(coords)]
curv = get_vts(dims, coords, {})
write_vts("test_comp.vts", curv)

# output cart space
lon_rad = np.radians(lon)
lat_rad = np.radians(lat)
coords_rad = get_mesh(r, lon_rad, lat_rad).reshape(-1, 3)

cart_coords = sph2car(coords_rad[:,0], coords_rad[:,1], coords_rad[:,2])
for i in range(len(cart_coords)):
  cart_coords[i] = cart_coords[i][..., None]
cart_coords = np.concatenate(cart_coords, axis=-1)
cart = get_vts(dims, cart_coords, {})
write_vts("test_cart.vts", cart)

meshe generated: (201, 360, 160, 3)


KeyboardInterrupt: 

In [6]:
# get spherical coords
coords = get_mesh(r, lon, lat)

# get cartesian coords
lon_rad = np.radians(lon)
lat_rad = np.radians(lat)
coords_rad = get_mesh(r, lon_rad, lat_rad)
cart_coords = sph2car(coords_rad[...,0], coords_rad[...,1], coords_rad[...,2])
for i in range(len(cart_coords)):
  cart_coords[i] = cart_coords[i][..., None]
cart_coords = np.concatenate(cart_coords, axis=-1)

meshe generated: (201, 360, 160, 3)
meshe generated: (201, 360, 160, 3)


In [7]:
coords_all = np.concatenate([coords, cart_coords], axis=-1)

In [8]:
coords_all.shape

(201, 360, 160, 6)

In [9]:
np.save("../data/earth_nopole.npy", np.array(coords_all, dtype=np.float32))

In [78]:
pvmi = cart_coords[:, -1].max()
pvma = cart_coords[:, -1].min()
display(pvmi, pvma)

6281.65089842488

-6281.650898424879

In [79]:
afmi = cart_coords[:, -1].max()
afma = cart_coords[:, -1].min()
display(afmi, afma)

6281.65089842488

-6281.650898424879

In [74]:
for sph, car in zip(coords[:10], cart_coords[:10]):
  print(f"{sph} --- {car}")

[3485.    0.    0.] --- [   0.    0. 3485.]
[3.48500000e+03 1.00278552e+00 0.00000000e+00] --- [   0.    0. 3485.]
[3.48500000e+03 2.00557103e+00 0.00000000e+00] --- [   0.    0. 3485.]
[3.48500000e+03 3.00835655e+00 0.00000000e+00] --- [   0.    0. 3485.]
[3485.            4.01114206    0.        ] --- [   0.    0. 3485.]
[3485.            5.01392758    0.        ] --- [   0.    0. 3485.]
[3485.            6.01671309    0.        ] --- [   0.    0. 3485.]
[3485.            7.01949861    0.        ] --- [   0.    0. 3485.]
[3485.            8.02228412    0.        ] --- [   0.    0. 3485.]
[3485.            9.02506964    0.        ] --- [   0.    0. 3485.]
[3485.           10.02785515    0.        ] --- [   0.    0. 3485.]
[3485.           11.03064067    0.        ] --- [   0.    0. 3485.]
[3485.           12.03342618    0.        ] --- [   0.    0. 3485.]
[3485.          13.0362117    0.       ] --- [   0.    0. 3485.]
[3485.           14.03899721    0.        ] --- [   0.    0. 3485.

KeyboardInterrupt: 

### Rooting Finding: phys -> comp

In [ ]:
def comp2phy(comp: np.ndarray):
  pass

def diff_phys(phys: np.ndarray, comp: np.ndarray):
  phys_est = comp2phy(comp)
  return phys - phys_est

op.root_scalar(diff_phys, method='newton')